# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # read the files
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # select year = 2018 and create a column 'W/L%'
    nhl_df = nhl_df.query('year == 2018')
    nhl_df = nhl_df[nhl_df['W'].str.isnumeric() & nhl_df['L'].str.isnumeric()].astype({'W': int, 'L':int}).reset_index(drop=True)
    nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    # split teams names in nhl_df
    nhl_df['team'] = nhl_df['team'].str.replace(r'\*+', '', regex=True)
    nhl_df['team'] = nhl_df['team'].str.split(' ').str[-1]
    
    # split teams names in cities
    team = {'RangersIslandersDevils':'Rangers Islanders Devils', 'KingsDucks':'Kings Ducks'}
    cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]', '', regex=True)
    cities['NHL'] = cities['NHL'].replace(team)
    cities['NHL'] = cities['NHL'].str.split(' ')
    cities = cities.explode('NHL')

    # merge nhl_df and cities
    df = pd.merge(nhl_df, cities, left_on='team', right_on='NHL')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL','W/L%']]

    # mean for same cities
    df_mean = df.groupby('Metropolitan area')[['W/L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL', 'W/L%_y']].rename(columns={'W/L%_y':'W/L%'})
    df = df.drop_duplicates('Metropolitan area').reset_index(drop=True)

    
    
    population_by_region = df['Population (2016 est.)[8]'].astype('int64') # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    corr = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return corr[0]

    raise NotImplementedError()

In [2]:
nhl_correlation()

0.012486162921209892

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # nba
    nba_df = nba_df.query('year == 2018')
    nba_df['team'] = nba_df['team'].str.replace(r'(\*?).\(.+\)', '', regex=True)
    nba_df['team'] = nba_df['team'].str.split(' ').str[-1]

    # cities
    teams = {'KnicksNets':'Knicks Nets', 'LakersClippers': 'Lakers Clippers'}
    cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]', '', regex=True)
    cities['NBA'] = cities['NBA'].replace(teams)
    cities['NBA'] = cities['NBA'].str.split(' ')
    cities = cities.explode('NBA')

    # merge
    df = pd.merge(nba_df, cities, left_on='team', right_on='NBA')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA','W/L%']].astype({'Population (2016 est.)[8]':'int64', 'W/L%':'float'})

    # mean and merge
    df_mean = df.groupby('Metropolitan area')[['W/L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA','W/L%_y']].rename(columns={'W/L%_y':'W/L%'})
    df = df.drop_duplicates('Metropolitan area')
    
        
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    corr = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr[0]
    raise NotImplementedError()

In [4]:
nba_correlation()

C:\Users\jeanm\AppData\Local\Temp\ipykernel_17960\2157696692.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_df['team'] = nba_df['team'].str.replace(r'(\*?).\(.+\)', '', regex=True)
C:\Users\jeanm\AppData\Local\Temp\ipykernel_17960\2157696692.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_df['team'] = nba_df['team'].str.split(' ').str[-1]


-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # mlb
    mlb_df = mlb_df.query('year == 2018')
    mlb_df = mlb_df.replace({'Boston Red Sox': 'RedSox', 'Chicago White Sox':'WhiteSox'})
    mlb_df['team'] = mlb_df['team'].str.split(' ').str[-1]
    
    # cities
    teams = {'YankeesMets':'Yankees Mets', 'DodgersAngels':'Dodgers Angels', 'GiantsAthletics':'Giants Athletics', 'CubsWhite Sox':'Cubs WhiteSox', 'Red Sox':'RedSox'}
    cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]', '', regex=True)
    cities['MLB'] = cities['MLB'].replace(teams).str.split(' ')
    cities = cities.explode('MLB')

    # merge
    df = pd.merge(mlb_df, cities, left_on='team', right_on='MLB')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB','W-L%']].astype({'Population (2016 est.)[8]':'int64', 'W-L%':'float'})

    # mean and merge
    df_mean = df.groupby('Metropolitan area')[['W-L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area').drop_duplicates('Metropolitan area').reset_index(drop=True)
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB','W-L%_y']].rename(columns={'W-L%_y': 'W-L%'})

    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()

In [6]:
mlb_correlation()

(0.15003737475409495, 0.46442827201123427)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    # nfl
    nfl_df = nfl_df.query('year == 2018')
    nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
    nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]

    # cities
    teams = {'GiantsJets':'Giants Jets', 'RamsChargers':'Rams Chargers', '49ersRaiders':'49ers Raiders'}
    cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]', '', regex=True)
    cities['NFL'] = cities['NFL'].replace(teams).str.split(' ')
    cities = cities.explode('NFL')

    # merge
    df = pd.merge(nfl_df, cities, left_on='team', right_on='NFL')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL','W-L%']].astype({'Population (2016 est.)[8]':'int64', 'W-L%':'float'})
    df_mean = df.groupby('Metropolitan area')[['W-L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area').drop_duplicates('Metropolitan area').reset_index(drop=True).rename(columns={'W-L%_y':'W-L%'})

    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    corr = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr[0]
    raise NotImplementedError()

In [8]:
nfl_correlation()

C:\Users\jeanm\AppData\Local\Temp\ipykernel_17960\2995283173.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
C:\Users\jeanm\AppData\Local\Temp\ipykernel_17960\2995283173.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]


0.00428214143639304

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values